In [14]:
# importing libraries which we will use in our code
import os # For file and directory operations
import numpy as np # For numerical operations
from scipy.io import wavfile # To read wav files
from python_speech_features import mfcc # For MFCC feature extraction
from sklearn.mixture import GaussianMixture # For GMM modelingC
import warnings # To handle warnings
warnings.filterwarnings("ignore") # Suppress warnings

In [15]:
# function for extracting features from audio file
def extract_feats(audio_path):
    freq,sig=wavfile.read(audio_path) # Read audio file
    feats=mfcc(sig,freq,winlen=0.025,winstep=0.01,numcep=20,nfilt=40,nfft=1200,preemph=0.97,ceplifter=22,appendEnergy=True)   # Extract MFCC features
    return feats # Returning extracted features

# function for training gmm model
def train_gmm(feats,n_comp=16):
    gmm=GaussianMixture(n_components=n_comp,covariance_type='full',max_iter=200,random_state=42) # Initializing GMM
    gmm.fit(feats)   # training GMM on features
    return gmm     #  Returning trained GMM

In [16]:
# function for loading the data
def load_train_data(base_path):     # base path is where the data is stored
    spkr_models={}      # Dictionary to store speaker models
    for spkr_folder in os.listdir(base_path):    # Iterating through speaker folders
        spkr_path=os.path.join(base_path,spkr_folder) # Getting full path of speaker folder
        if os.path.isdir(spkr_path):     # Checking if it's a directory
            all_feats=[]       # List to store all features for a speaker
            for audio_file in os.listdir(spkr_path): # Iterating through audio files
                if audio_file.endswith('.wav'):      # Checking if it's a wav file
                    audio_path=os.path.join(spkr_path,audio_file)    # Getting full path of audio file
                    feats=extract_feats(audio_path) # extracting the features from audio
                    all_feats.extend(feats)      # Adding features to list
            if all_feats:        # If all features were extracted
                print(f"Training model for {spkr_folder}") # Printing progress
                spkr_models[spkr_folder]=train_gmm(np.array(all_feats)) # Train GMM and store in dictionary
    return spkr_models # Returning dictionary of trained models


In [17]:
# function to identify speaker
def identify_spkr(audio_path,spkr_models):
    feats=extract_feats(audio_path)     # Extracting features from test audio
    max_score=float('-inf')      # Initializing max score as low
    id_spkr=None           # Initializing identified speaker as none
    for spkr,model in spkr_models.items():       # Iterating through all speaker models
        score=np.mean(model.score_samples(feats))       # Calculating score for current model
        if score>max_score:         # If score is higher than max score
            max_score=score        # Updating max score
            id_spkr=spkr         # Updating identified speaker
    return id_spkr           # Returning identified speaker

In [18]:
train_path="C:\\Users\\ajayd\\Desktop\\Assignment\\Speaker_data\\Voice_Samples_Training"     # Path to training data
test_path="C:\\Users\\ajayd\\Desktop\\Assignment\\Speaker_data\\Testing_Audio"      # Path to test data

In [19]:
print("Training speaker models...")      # Printing progress
spkr_models=load_train_data(train_path)     # Loading and training speaker models
print("\nTesting the system...") 
named_results={}       # Dictionary to store results for named samples
unnamed_results={}        # Dictionary to store results for unnamed samples
for audio_file in os.listdir(test_path): # Iterating through test audio files
    if not audio_file.endswith('.wav'):     # Skiping non-wav files
        continue
    audio_path=os.path.join(test_path,audio_file)      # Getting full path of test audio
    id_spkr=identify_spkr(audio_path,spkr_models)       # Identifying speaker
    print(f"Audio file: {audio_file}")             #Printing audio file name
    print(f"Identified speaker: {id_spkr}")           #Print identified speaker
    if audio_file.startswith('P') and audio_file[1].isdigit():           # If it's an unnamed sample (P1-P6)
        unnamed_results[audio_file]={'id_spkr':id_spkr}              # Storing result in unnamed_results
    else:                                          # If it's a named sample
        true_spkr=audio_file.split('_')[0]                 # Get true speaker from filename
        named_results[audio_file]={'true_spkr':true_spkr,'id_spkr':id_spkr,'correct':true_spkr in id_spkr}         # Storing result in named_results
# Calculate overall accuracy for named samples
total_named=len(named_results)                # Total number of named samples
correct_named=sum(1 for r in named_results.values() if r['correct'])          #Number of correct identifications
named_accuracy=(correct_named/total_named)*100                   # Calculating accuracy percentage

print(f"\nOverall Accuracy: {named_accuracy:.2f}%")           # Printing overall accuracy

print("\nUnnamed Samples Identification (P1-P6):")         # Printing unnamed samples
for file,data in sorted(unnamed_results.items()):            # Iterating through unnamed results
    print(f"{file}: Identified as {data['id_spkr']}")            # Printing identification for unnamed sample


Training speaker models...
Training model for Abhay
Training model for Eknath
Training model for Rg
Training model for Rishika
Training model for ShivamY
Training model for Vaibhav

Testing the system...
Audio file: Abhay_15.wav
Identified speaker: Abhay
Audio file: Abhay_16.wav
Identified speaker: Abhay
Audio file: Abhay_17.wav
Identified speaker: Abhay
Audio file: Abhay_18.wav
Identified speaker: Abhay
Audio file: Abhay_19.wav
Identified speaker: Abhay
Audio file: Abhay_audio1.wav
Identified speaker: Abhay
Audio file: Abhay_audio2.wav
Identified speaker: Abhay
Audio file: Abhay_audio3.wav
Identified speaker: Abhay
Audio file: Abhay_audio4.wav
Identified speaker: Abhay
Audio file: Abhay_audio5.wav
Identified speaker: Abhay
Audio file: Eknath_audio1.wav
Identified speaker: Eknath
Audio file: Eknath_audio2.wav
Identified speaker: Eknath
Audio file: Eknath_audio3.wav
Identified speaker: Eknath
Audio file: Eknath_audio4.wav
Identified speaker: Eknath
Audio file: Eknath_audio5.wav
Identifi